Building and Populating Silver Layer

In [ ]:
USE ROLE ROLE_TEAM_HNP;
USE DATABASE DB_TEAM_HNP;
USE SCHEMA BRONZE;
USE WAREHOUSE ANIMAL_TASK_WH;

-- ============================================================================
-- CREATE REFERENCE TABLES (DIMENSION TABLES)
-- ============================================================================

-- Create Medicine Reference Table with auto-incrementing ID
CREATE OR REPLACE TABLE Silver.DrugDatabase_Medicine_Ref AS
SELECT 
    ROW_NUMBER() OVER (ORDER BY Medicine_Name) AS Medicine_ID,
    Medicine_Name
FROM (
    SELECT DISTINCT Medicine_Name 
    FROM Bronze.DrugDatabase_Raw
)
ORDER BY Medicine_ID;

-- Create Manufacturer Reference Table with auto-incrementing ID
CREATE OR REPLACE TABLE Silver.DrugDatabase_Manufacturer_Ref AS
SELECT 
    ROW_NUMBER() OVER (ORDER BY Manufacturer) AS Manufacturer_ID,
    Manufacturer
FROM (
    SELECT DISTINCT Manufacturer 
    FROM Bronze.DrugDatabase_Raw
    WHERE Manufacturer IS NOT NULL
)
ORDER BY Manufacturer_ID;

-- ============================================================================
-- CREATE SILVER CHILD TABLES (with Medicine_ID references)
-- ============================================================================

-- Create Image_URL child table (with Medicine_ID)
CREATE OR REPLACE TABLE Silver.DrugDatabase_ImageURL AS
SELECT 
    m.Medicine_ID,
    r.Image_URL
FROM Bronze.DrugDatabase_Raw r
JOIN Silver.DrugDatabase_Medicine_Ref m ON r.Medicine_Name = m.Medicine_Name
ORDER BY m.Medicine_ID;

-- Create Manufacturer child table (with Medicine_ID and Manufacturer_ID)
CREATE OR REPLACE TABLE Silver.DrugDatabase_Manufacturer AS
SELECT 
    m.Medicine_ID,
    mf.Manufacturer_ID
FROM Bronze.DrugDatabase_Raw r
JOIN Silver.DrugDatabase_Medicine_Ref m ON r.Medicine_Name = m.Medicine_Name
JOIN Silver.DrugDatabase_Manufacturer_Ref mf ON r.Manufacturer = mf.Manufacturer
ORDER BY m.Medicine_ID;

-- Create Reviews child table (with Medicine_ID)
CREATE OR REPLACE TABLE Silver.DrugDatabase_Reviews AS
SELECT 
    m.Medicine_ID,
    r.Excellent_Review_Percentage,
    r.Average_Review_Percentage,
    r.Poor_Review_Percentage
FROM Bronze.DrugDatabase_Raw r
JOIN Silver.DrugDatabase_Medicine_Ref m ON r.Medicine_Name = m.Medicine_Name
ORDER BY m.Medicine_ID;

-- ============================================================================
-- CREATE AND NORMALIZE COMPOSITION TABLE
-- ============================================================================

-- Step 1: Create unnormalized Composition table
CREATE OR REPLACE TABLE Silver.DrugDatabase_Composition AS
SELECT 
    m.Medicine_ID,
    r.Composition
FROM Bronze.DrugDatabase_Raw r
JOIN Silver.DrugDatabase_Medicine_Ref m ON r.Medicine_Name = m.Medicine_Name
ORDER BY m.Medicine_ID;

-- Step 2: Normalize Composition table in place
CREATE OR REPLACE TABLE Silver.DrugDatabase_Composition AS
WITH cleaned_composition AS (
    SELECT 
        Medicine_ID,
        TRIM(Composition) AS composition_trimmed
    FROM Silver.DrugDatabase_Composition
),
split_ingredients AS (
    SELECT 
        Medicine_ID,
        TRIM(value) AS ingredient_with_dosage
    FROM cleaned_composition,
    LATERAL FLATTEN(input => SPLIT(composition_trimmed, '+'))
),
parsed_ingredients AS (
    SELECT 
        Medicine_ID,
        TRIM(REGEXP_SUBSTR(ingredient_with_dosage, '^[^(]+')) AS Ingredient_Name,
        TRIM(REGEXP_REPLACE(REGEXP_SUBSTR(ingredient_with_dosage, '\\([^)]+\\)'), '[()]', '')) AS Dosage
    FROM split_ingredients
    WHERE ingredient_with_dosage IS NOT NULL 
        AND TRIM(ingredient_with_dosage) != ''
)
SELECT 
    Medicine_ID,
    Ingredient_Name,
    Dosage
FROM parsed_ingredients
WHERE Ingredient_Name IS NOT NULL 
    AND Ingredient_Name != ''
ORDER BY Medicine_ID, Ingredient_Name;

-- ============================================================================
-- CREATE AND NORMALIZE USES TABLE
-- ============================================================================

-- Step 1: Create unnormalized Uses table
CREATE OR REPLACE TABLE Silver.DrugDatabase_Uses AS
SELECT 
    m.Medicine_ID,
    r.Uses
FROM Bronze.DrugDatabase_Raw r
JOIN Silver.DrugDatabase_Medicine_Ref m ON r.Medicine_Name = m.Medicine_Name
ORDER BY m.Medicine_ID;

-- Step 2: Normalize Uses table in place
CREATE OR REPLACE TABLE Silver.DrugDatabase_Uses AS
WITH cleaned_uses AS (
    SELECT 
        Medicine_ID,
        TRIM(Uses) AS uses_trimmed
    FROM Silver.DrugDatabase_Uses
),
parentheses_removed AS (
    SELECT 
        Medicine_ID,
        REGEXP_REPLACE(uses_trimmed, '\\([^)]*\\)', ' ') AS uses_no_parens
    FROM cleaned_uses
),
spacing_fixed AS (
    SELECT 
        Medicine_ID,
        REGEXP_REPLACE(uses_no_parens, '([a-z])([A-Z])', '\\1 \\2') AS uses_with_spaces
    FROM parentheses_removed
),
structured_extracted AS (
    SELECT 
        Medicine_ID,
        uses_with_spaces,
        REGEXP_SUBSTR_ALL(
            uses_with_spaces, 
            '(Treatment|Prevention|Management|Relief|Control|Reduction|Cure) of [^A-Z]*'
        ) AS structured_phrases,
        REGEXP_REPLACE(
            uses_with_spaces,
            '(Treatment|Prevention|Management|Relief|Control|Reduction|Cure) of [^A-Z]*',
            ''
        ) AS remaining_text
    FROM spacing_fixed
),
remaining_split AS (
    SELECT 
        Medicine_ID,
        structured_phrases,
        SPLIT(TRIM(remaining_text), ' ') AS remaining_words
    FROM structured_extracted
),
grouped_words AS (
    SELECT 
        Medicine_ID,
        structured_phrases,
        ARRAY_AGG(
            CASE 
                WHEN value RLIKE '^[A-Z].*' THEN '|||' || value
                ELSE value
            END
        ) WITHIN GROUP (ORDER BY index) AS word_markers
    FROM remaining_split,
    LATERAL FLATTEN(input => remaining_words)
    GROUP BY Medicine_ID, structured_phrases
),
rejoined AS (
    SELECT 
        Medicine_ID,
        structured_phrases,
        SPLIT(ARRAY_TO_STRING(word_markers, ' '), '|||') AS split_phrases
    FROM grouped_words
),
all_phrases AS (
    SELECT 
        Medicine_ID,
        ARRAY_CAT(
            COALESCE(structured_phrases, ARRAY_CONSTRUCT()),
            COALESCE(split_phrases, ARRAY_CONSTRUCT())
        ) AS use_array
    FROM rejoined
),
flattened AS (
    SELECT 
        Medicine_ID,
        TRIM(value) AS use_item
    FROM all_phrases,
    LATERAL FLATTEN(input => use_array)
    WHERE value IS NOT NULL
),
cleaned AS (
    SELECT 
        Medicine_ID,
        use_item
    FROM flattened
    WHERE LENGTH(use_item) > 3
        AND use_item NOT RLIKE '^(Treatment|Prevention|Management|Relief|Control|Reduction|Cure) of$'
        AND use_item != ''
        AND TRIM(use_item) != ''
)
SELECT DISTINCT
    Medicine_ID,
    use_item AS Uses
FROM cleaned
ORDER BY Medicine_ID, Uses;

-- ============================================================================
-- CREATE AND NORMALIZE SIDE_EFFECTS TABLE
-- ============================================================================

-- Step 1: Create unnormalized Side_Effects table
CREATE OR REPLACE TABLE Silver.DrugDatabase_SideEffects AS
SELECT 
    m.Medicine_ID,
    r.Side_Effects
FROM Bronze.DrugDatabase_Raw r
JOIN Silver.DrugDatabase_Medicine_Ref m ON r.Medicine_Name = m.Medicine_Name
ORDER BY m.Medicine_ID;

-- Step 2: Normalize Side_Effects table in place
CREATE OR REPLACE TABLE Silver.DrugDatabase_SideEffects AS
WITH cleaned_side_effects AS (
    SELECT 
        Medicine_ID,
        TRIM(Side_Effects) AS side_effects_trimmed
    FROM Silver.DrugDatabase_SideEffects
),
parentheses_removed AS (
    SELECT 
        Medicine_ID,
        REGEXP_REPLACE(side_effects_trimmed, '\\([^)]*\\)', ' ') AS side_effects_no_parens
    FROM cleaned_side_effects
),
spacing_fixed AS (
    SELECT 
        Medicine_ID,
        REGEXP_REPLACE(side_effects_no_parens, '([a-z])([A-Z])', '\\1 \\2') AS side_effects_with_spaces
    FROM parentheses_removed
),
words_split AS (
    SELECT 
        Medicine_ID,
        SPLIT(side_effects_with_spaces, ' ') AS words
    FROM spacing_fixed
),
grouped_words AS (
    SELECT 
        Medicine_ID,
        ARRAY_AGG(
            CASE 
                WHEN value RLIKE '^[A-Z].*' THEN '|||' || value
                ELSE value
            END
        ) WITHIN GROUP (ORDER BY index) AS word_markers
    FROM words_split,
    LATERAL FLATTEN(input => words)
    GROUP BY Medicine_ID
),
rejoined AS (
    SELECT 
        Medicine_ID,
        SPLIT(ARRAY_TO_STRING(word_markers, ' '), '|||') AS side_effect_phrases
    FROM grouped_words
),
flattened AS (
    SELECT 
        Medicine_ID,
        TRIM(value) AS side_effect_item
    FROM rejoined,
    LATERAL FLATTEN(input => side_effect_phrases)
    WHERE value IS NOT NULL
),
cleaned AS (
    SELECT 
        Medicine_ID,
        side_effect_item
    FROM flattened
    WHERE LENGTH(side_effect_item) > 2
        AND side_effect_item != ''
        AND TRIM(side_effect_item) != ''
)
SELECT DISTINCT
    Medicine_ID,
    side_effect_item AS Side_Effects
FROM cleaned
ORDER BY Medicine_ID, Side_Effects;


In [ ]:
USE ROLE ROLE_TEAM_HNP;
USE WAREHOUSE ANIMAL_TASK_WH;
USE DATABASE DB_TEAM_HNP;
USE SCHEMA GOLD;

Which manufacturers produce the highest-quality medicines based on customer reviews?

In [ ]:
-- =====================================================================
-- GOLD TABLE 1: Manufacturer Performance Summary
-- Business Use Case 1: Manufacturer Performance Analysis
-- =====================================================================
CREATE OR REPLACE DYNAMIC TABLE GOLD.DM_MANUFACTURER_PERFORMANCE
TARGET_LAG = '1 HOUR'
WAREHOUSE = ANIMAL_TASK_WH
AS
SELECT 
    mfr.MANUFACTURER,  
    COUNT(DISTINCT m.MEDICINE_ID) AS TOTAL_MEDICINES,
    ROUND(AVG(r.EXCELLENT_REVIEW_PERCENTAGE), 2) AS AVG_EXCELLENT_REVIEW_PCT,
    ROUND(AVG(r.AVERAGE_REVIEW_PERCENTAGE), 2) AS AVG_AVERAGE_REVIEW_PCT,
    ROUND(AVG(r.POOR_REVIEW_PERCENTAGE), 2) AS AVG_POOR_REVIEW_PCT,
    ROUND(MAX(r.EXCELLENT_REVIEW_PERCENTAGE), 2) AS MAX_EXCELLENT_REVIEW_PCT,
    ROUND(MIN(r.EXCELLENT_REVIEW_PERCENTAGE), 2) AS MIN_EXCELLENT_REVIEW_PCT,
    -- Overall Quality Score (Weighted: Excellent=1, Average=0.5, Poor=0)
    ROUND(
        (AVG(r.EXCELLENT_REVIEW_PERCENTAGE) * 1.0 + 
         AVG(r.AVERAGE_REVIEW_PERCENTAGE) * 0.5 + 
         AVG(r.POOR_REVIEW_PERCENTAGE) * 0.0) / 100,
        4
    ) AS OVERALL_QUALITY_SCORE,
    -- Count of high-performing medicines (Excellent > 40%)
    COUNT_IF(r.EXCELLENT_REVIEW_PERCENTAGE > 40) AS HIGH_PERFORMING_MEDICINE_COUNT,
    -- Count of low-performing medicines (Poor > 30%)
    COUNT_IF(r.POOR_REVIEW_PERCENTAGE > 30) AS LOW_PERFORMING_MEDICINE_COUNT
FROM SILVER.DRUGDATABASE_MANUFACTURER m
JOIN SILVER.DRUGDATABASE_MANUFACTURER_REF mfr 
    ON m.MANUFACTURER_ID = mfr.MANUFACTURER_ID
JOIN SILVER.DRUGDATABASE_REVIEWS r 
    ON m.MEDICINE_ID = r.MEDICINE_ID
GROUP BY mfr.MANUFACTURER
ORDER BY AVG_EXCELLENT_REVIEW_PCT DESC;

Which specific medicines deliver the best customer satisfaction and should be promoted?

In [ ]:
-- =====================================================================
-- GOLD TABLE 2: Top-Performing Medicines Analysis
-- Business Use Case 2: Top-Performing Medicines Identification
-- =====================================================================
CREATE OR REPLACE DYNAMIC TABLE GOLD.DM_TOP_MEDICINES
TARGET_LAG = '1 HOUR'
WAREHOUSE = ANIMAL_TASK_WH
AS
SELECT 
    med.MEDICINE_NAME,  
    mfr.MANUFACTURER,   
    r.EXCELLENT_REVIEW_PERCENTAGE,
    r.AVERAGE_REVIEW_PERCENTAGE,
    r.POOR_REVIEW_PERCENTAGE,
    -- Overall Quality Score (Weighted)
    ROUND(
        (r.EXCELLENT_REVIEW_PERCENTAGE * 1.0 + 
         r.AVERAGE_REVIEW_PERCENTAGE * 0.5) / 100,
        4
    ) AS OVERALL_QUALITY_SCORE,
    -- Customer Satisfaction Score (Simple: Excellent - Poor)
    (r.EXCELLENT_REVIEW_PERCENTAGE - r.POOR_REVIEW_PERCENTAGE) AS SATISFACTION_SCORE,
    -- Number of unique ingredients
    (SELECT COUNT(DISTINCT INGREDIENT_NAME) 
     FROM SILVER.DRUGDATABASE_COMPOSITION c 
     WHERE c.MEDICINE_ID = med.MEDICINE_ID) AS INGREDIENT_COUNT,
    -- Number of uses
    (SELECT COUNT(*) 
     FROM SILVER.DRUGDATABASE_USES u 
     WHERE u.MEDICINE_ID = med.MEDICINE_ID) AS USE_CASE_COUNT,
    -- Number of side effects
    (SELECT COUNT(*) 
     FROM SILVER.DRUGDATABASE_SIDEEFFECTS s 
     WHERE s.MEDICINE_ID = med.MEDICINE_ID) AS SIDE_EFFECT_COUNT,
    -- Image URL for display
    img.IMAGE_URL,
    -- Performance Category
    CASE 
        WHEN r.EXCELLENT_REVIEW_PERCENTAGE >= 50 THEN 'Excellent Performer'
        WHEN r.EXCELLENT_REVIEW_PERCENTAGE >= 35 THEN 'Good Performer'
        WHEN r.EXCELLENT_REVIEW_PERCENTAGE >= 20 THEN 'Average Performer'
        ELSE 'Below Average'
    END AS PERFORMANCE_CATEGORY,
    -- Ranking within all medicines
    RANK() OVER (ORDER BY r.EXCELLENT_REVIEW_PERCENTAGE DESC) AS OVERALL_RANK
FROM SILVER.DRUGDATABASE_MEDICINE_REF med
JOIN SILVER.DRUGDATABASE_REVIEWS r 
    ON med.MEDICINE_ID = r.MEDICINE_ID
JOIN SILVER.DRUGDATABASE_MANUFACTURER m 
    ON med.MEDICINE_ID = m.MEDICINE_ID
JOIN SILVER.DRUGDATABASE_MANUFACTURER_REF mfr 
    ON m.MANUFACTURER_ID = mfr.MANUFACTURER_ID
LEFT JOIN SILVER.DRUGDATABASE_IMAGEURL img 
    ON med.MEDICINE_ID = img.MEDICINE_ID
ORDER BY OVERALL_QUALITY_SCORE DESC;

Which active ingredients are most commonly used and how do medicines with specific ingredients perform?

In [ ]:
-- =====================================================================
-- GOLD TABLE 3: Ingredient Performance Analysis
-- Business Use Case 3: Composition/Ingredient Analysis
-- =====================================================================
CREATE OR REPLACE DYNAMIC TABLE GOLD.DM_INGREDIENT_ANALYSIS
TARGET_LAG = '1 HOUR'
WAREHOUSE = ANIMAL_TASK_WH
AS
SELECT
    c.INGREDIENT_NAME,  -- :white_check_mark: Already business-friendly
    c.DOSAGE,           -- :white_check_mark: Already business-friendly
    COUNT(DISTINCT c.MEDICINE_ID) AS MEDICINE_COUNT,
    -- Average reviews for medicines containing this ingredient
    ROUND(AVG(r.EXCELLENT_REVIEW_PERCENTAGE), 2) AS AVG_EXCELLENT_REVIEW_PCT,
    ROUND(AVG(r.AVERAGE_REVIEW_PERCENTAGE), 2) AS AVG_AVERAGE_REVIEW_PCT,
    ROUND(AVG(r.POOR_REVIEW_PERCENTAGE), 2) AS AVG_POOR_REVIEW_PCT,
    -- Quality score for this ingredient
    ROUND(
        (AVG(r.EXCELLENT_REVIEW_PERCENTAGE) * 1.0 +
         AVG(r.AVERAGE_REVIEW_PERCENTAGE) * 0.5) / 100,
        4
    ) AS INGREDIENT_QUALITY_SCORE,
    -- Best performing medicine with this ingredient (NAME not ID)
    MAX_BY(med.MEDICINE_NAME, r.EXCELLENT_REVIEW_PERCENTAGE) AS BEST_MEDICINE,
    MAX(r.EXCELLENT_REVIEW_PERCENTAGE) AS BEST_MEDICINE_REVIEW_PCT,
    -- Count of high-performing medicines with this ingredient
    COUNT_IF(r.EXCELLENT_REVIEW_PERCENTAGE > 40) AS HIGH_PERFORMING_COUNT,
    -- Most common manufacturer using this ingredient (NAME not ID)
    MODE(mfr.MANUFACTURER) AS MOST_COMMON_MANUFACTURER,
    -- Popularity rank
    RANK() OVER (ORDER BY COUNT(DISTINCT c.MEDICINE_ID) DESC) AS POPULARITY_RANK
FROM SILVER.DRUGDATABASE_COMPOSITION c
JOIN SILVER.DRUGDATABASE_REVIEWS r
    ON c.MEDICINE_ID = r.MEDICINE_ID
JOIN SILVER.DRUGDATABASE_MANUFACTURER m
    ON c.MEDICINE_ID = m.MEDICINE_ID
JOIN SILVER.DRUGDATABASE_MANUFACTURER_REF mfr
    ON m.MANUFACTURER_ID = mfr.MANUFACTURER_ID
JOIN SILVER.DRUGDATABASE_MEDICINE_REF med
    ON c.MEDICINE_ID = med.MEDICINE_ID
GROUP BY c.INGREDIENT_NAME, c.DOSAGE
ORDER BY MEDICINE_COUNT DESC;

The snowflake streamlit app for visualizations from these gold tables are unable to be directly ran in a notebook and are at this link: https://app.snowflake.com/sfedu02/aeb73262/#/streamlit-apps/DB_TEAM_HNP.GOLD.VBTDS42P_JGT2JJ5 . The commented code for this app is below:

In [ ]:
"""
MIS 381N Group Project: Medicine Data Warehouse Dashboard
Team HNP - Streamlit Dashboard with 3 Gold Layer Visualizations
"""
'''
import streamlit as st
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from snowflake.snowpark.context import get_active_session

# =====================================================================
# PAGE CONFIGURATION
# =====================================================================

st.set_page_config(
    page_title="Medicine Analytics Dashboard",
    page_icon="💊",
    layout="wide"
)

# =====================================================================
# SNOWFLAKE CONNECTION (Automatic in Snowflake Streamlit)
# =====================================================================

# Get the current Snowflake session (no credentials needed!)
session = get_active_session()

# =====================================================================
# DATA LOADING FUNCTIONS
# =====================================================================

@st.cache_data
def load_manufacturer_performance():
    """Load manufacturer performance data from Gold layer"""
    query = """
    SELECT 
        MANUFACTURER,
        TOTAL_MEDICINES,
        AVG_EXCELLENT_REVIEW_PCT,
        AVG_AVERAGE_REVIEW_PCT,
        AVG_POOR_REVIEW_PCT,
        OVERALL_QUALITY_SCORE,
        HIGH_PERFORMING_MEDICINE_COUNT
    FROM DB_TEAM_HNP.GOLD.DM_MANUFACTURER_PERFORMANCE
    WHERE TOTAL_MEDICINES >= 5
    ORDER BY OVERALL_QUALITY_SCORE DESC
    LIMIT 20
    """
    return session.sql(query).to_pandas()

@st.cache_data
def load_top_medicines():
    """Load top performing medicines from Gold layer"""
    query = """
    SELECT 
        MEDICINE_NAME,
        MANUFACTURER,
        EXCELLENT_REVIEW_PERCENTAGE,
        OVERALL_QUALITY_SCORE,
        PERFORMANCE_CATEGORY,
        INGREDIENT_COUNT,
        SIDE_EFFECT_COUNT,
        OVERALL_RANK
    FROM DB_TEAM_HNP.GOLD.DM_TOP_MEDICINES
    WHERE OVERALL_RANK <= 50
    ORDER BY OVERALL_RANK
    """
    return session.sql(query).to_pandas()

@st.cache_data
def load_ingredient_analysis():
    """Load ingredient analysis from Gold layer"""
    query = """
    SELECT 
        INGREDIENT_NAME,
        DOSAGE,
        MEDICINE_COUNT,
        AVG_EXCELLENT_REVIEW_PCT,
        INGREDIENT_QUALITY_SCORE,
        BEST_MEDICINE,
        POPULARITY_RANK
    FROM DB_TEAM_HNP.GOLD.DM_INGREDIENT_ANALYSIS
    WHERE POPULARITY_RANK <= 30
    ORDER BY POPULARITY_RANK
    """
    return session.sql(query).to_pandas()

# =====================================================================
# MAIN APPLICATION
# =====================================================================

# Title and description
st.title("💊 Medicine Data Warehouse Analytics Dashboard")
st.markdown("### MIS 381N Group Project - Team HNP")
st.markdown("---")

# Load all data
with st.spinner("Loading data from Gold layer..."):
    manufacturer_df = load_manufacturer_performance()
    top_medicines_df = load_top_medicines()
    ingredient_df = load_ingredient_analysis()

# Sidebar info
st.sidebar.markdown("## 📊 Dashboard Metrics")
st.sidebar.metric("Manufacturers Analyzed", len(manufacturer_df))
st.sidebar.metric("Top Medicines", len(top_medicines_df))
st.sidebar.metric("Ingredients Tracked", len(ingredient_df))

st.sidebar.markdown("---")
st.sidebar.markdown("## 📋 Gold Layer Tables")
st.sidebar.markdown("✅ DM_MANUFACTURER_PERFORMANCE")
st.sidebar.markdown("✅ DM_TOP_MEDICINES")
st.sidebar.markdown("✅ DM_INGREDIENT_ANALYSIS")

# ================================================================
# VISUALIZATION 1: Manufacturer Performance Analysis
# ================================================================

st.header("📊 Visualization 1: Manufacturer Performance Analysis")
st.markdown("**Business Question**: Which manufacturers produce the highest-quality medicines?")
st.markdown("**Data Source**: `GOLD.DM_MANUFACTURER_PERFORMANCE`")

# Create tabs for different views
tab1, tab2 = st.tabs(["📈 Quality Score Ranking", "📊 Review Distribution"])

with tab1:
    # Bar chart: Top manufacturers by quality score
    fig1 = go.Figure()
    
    fig1.add_trace(go.Bar(
        x=manufacturer_df['MANUFACTURER'],
        y=manufacturer_df['OVERALL_QUALITY_SCORE'],
        text=manufacturer_df['OVERALL_QUALITY_SCORE'].round(3),
        textposition='outside',
        marker_color=manufacturer_df['OVERALL_QUALITY_SCORE'],
        marker_colorscale='Greens',
        hovertemplate='<b>%{x}</b><br>' +
                     'Quality Score: %{y:.4f}<br>' +
                     '<extra></extra>'
    ))
    
    fig1.update_layout(
        title='Top 20 Manufacturers by Overall Quality Score',
        xaxis_title='Manufacturer',
        yaxis_title='Quality Score (0-1 scale)',
        height=500,
        xaxis_tickangle=-45,
        showlegend=False
    )
    
    st.plotly_chart(fig1, use_container_width=True)
    
    # Key insights
    col1, col2, col3 = st.columns(3)
    with col1:
        best_manufacturer = manufacturer_df.iloc[0]
        st.metric(
            "🏆 Best Manufacturer", 
            best_manufacturer['MANUFACTURER'],
            f"Score: {best_manufacturer['OVERALL_QUALITY_SCORE']:.3f}"
        )
    with col2:
        avg_score = manufacturer_df['OVERALL_QUALITY_SCORE'].mean()
        st.metric(
            "📊 Average Quality Score",
            f"{avg_score:.3f}",
            "Across top 20"
        )
    with col3:
        total_medicines = manufacturer_df['TOTAL_MEDICINES'].sum()
        st.metric(
            "💊 Total Medicines",
            f"{total_medicines:,}",
            "From top 20 manufacturers"
        )

with tab2:
    # Stacked bar chart: Review distribution
    fig2 = go.Figure()
    
    fig2.add_trace(go.Bar(
        name='Excellent Reviews',
        x=manufacturer_df['MANUFACTURER'],
        y=manufacturer_df['AVG_EXCELLENT_REVIEW_PCT'],
        marker_color='green'
    ))
    
    fig2.add_trace(go.Bar(
        name='Average Reviews',
        x=manufacturer_df['MANUFACTURER'],
        y=manufacturer_df['AVG_AVERAGE_REVIEW_PCT'],
        marker_color='orange'
    ))
    
    fig2.add_trace(go.Bar(
        name='Poor Reviews',
        x=manufacturer_df['MANUFACTURER'],
        y=manufacturer_df['AVG_POOR_REVIEW_PCT'],
        marker_color='red'
    ))
    
    fig2.update_layout(
        barmode='stack',
        title='Review Distribution by Manufacturer',
        xaxis_title='Manufacturer',
        yaxis_title='Average Review Percentage',
        height=500,
        xaxis_tickangle=-45
    )
    
    st.plotly_chart(fig2, use_container_width=True)

st.markdown("---")

# ================================================================
# VISUALIZATION 2: Top-Performing Medicines
# ================================================================

st.header("🏆 Visualization 2: Top-Performing Medicines")
st.markdown("**Business Question**: Which medicines deliver the best customer satisfaction?")
st.markdown("**Data Source**: `GOLD.DM_TOP_MEDICINES`")

# Filter by performance category
col1, col2 = st.columns([1, 3])
with col1:
    categories = ['All'] + list(top_medicines_df['PERFORMANCE_CATEGORY'].unique())
    selected_category = st.selectbox("Filter by Performance", categories)

# Filter data
if selected_category != 'All':
    filtered_medicines = top_medicines_df[
        top_medicines_df['PERFORMANCE_CATEGORY'] == selected_category
    ]
else:
    filtered_medicines = top_medicines_df

# Create tabs
tab1, tab2 = st.tabs(["📋 Leaderboard Table", "📈 Performance vs Complexity"])

with tab1:
    # Display as formatted table
    display_df = filtered_medicines[[
        'OVERALL_RANK', 'MEDICINE_NAME', 'MANUFACTURER', 
        'EXCELLENT_REVIEW_PERCENTAGE', 'OVERALL_QUALITY_SCORE',
        'PERFORMANCE_CATEGORY'
    ]].head(20)
    
    formatted_df = display_df.copy()
    formatted_df['EXCELLENT_REVIEW_PERCENTAGE'] = formatted_df['EXCELLENT_REVIEW_PERCENTAGE'].apply(lambda x: f"{x:.1f}%")
    formatted_df['OVERALL_QUALITY_SCORE'] = formatted_df['OVERALL_QUALITY_SCORE'].apply(lambda x: f"{x:.4f}")
    
    st.dataframe(
        formatted_df,
        use_container_width=True,
        height=600
    )
    
    
    # Download button
    csv = filtered_medicines.to_csv(index=False)
    st.download_button(
        label="📥 Download Full Data as CSV",
        data=csv,
        file_name="top_medicines.csv",
        mime="text/csv"
    )

with tab2:
    # Scatter plot: Quality vs Complexity
    fig3 = px.scatter(
        filtered_medicines.head(30),
        x='INGREDIENT_COUNT',
        y='EXCELLENT_REVIEW_PERCENTAGE',
        size='OVERALL_QUALITY_SCORE',
        color='PERFORMANCE_CATEGORY',
        hover_data=['MEDICINE_NAME', 'MANUFACTURER'],
        title='Medicine Performance vs Ingredient Complexity (Top 30)',
        labels={
            'INGREDIENT_COUNT': 'Number of Ingredients',
            'EXCELLENT_REVIEW_PERCENTAGE': 'Excellent Review %'
        },
        color_discrete_map={
            'Excellent Performer': 'green',
            'Good Performer': 'lightgreen',
            'Average Performer': 'orange',
            'Below Average': 'red'
        }
    )
    
    fig3.update_layout(height=500)
    st.plotly_chart(fig3, use_container_width=True)
    
    st.markdown("**💡 Insight**: This chart shows if medicine complexity (number of ingredients) correlates with performance.")

st.markdown("---")

# ================================================================
# VISUALIZATION 3: Ingredient Analysis
# ================================================================

st.header("🧪 Visualization 3: Ingredient Popularity & Performance")
st.markdown("**Business Question**: Which ingredients are most common and how do they perform?")
st.markdown("**Data Source**: `GOLD.DM_INGREDIENT_ANALYSIS`")

# Create tabs
tab1, tab2 = st.tabs(["📊 Popularity Ranking", "⭐ Quality Analysis"])

with tab1:
    # Horizontal bar chart: Most popular ingredients
    fig4 = go.Figure()
    
    top_15_ingredients = ingredient_df.head(15)
    
    fig4.add_trace(go.Bar(
        x=top_15_ingredients['MEDICINE_COUNT'],
        y=top_15_ingredients['INGREDIENT_NAME'],
        orientation='h',
        text=top_15_ingredients['MEDICINE_COUNT'],
        textposition='outside',
        marker_color=top_15_ingredients['MEDICINE_COUNT'],
        marker_colorscale='Blues',
        hovertemplate='<b>%{y}</b><br>' +
                     'Used in %{x} medicines<br>' +
                     '<extra></extra>'
    ))
    
    fig4.update_layout(
        title='Top 15 Most Popular Ingredients',
        xaxis_title='Number of Medicines',
        yaxis_title='Ingredient',
        height=600,
        yaxis={'categoryorder': 'total ascending'}
    )
    
    st.plotly_chart(fig4, use_container_width=True)
    
    # Key metrics
    col1, col2, col3 = st.columns(3)
    with col1:
        most_popular = ingredient_df.iloc[0]
        st.metric(
            "🥇 Most Popular Ingredient",
            most_popular['INGREDIENT_NAME'],
            f"{most_popular['MEDICINE_COUNT']} medicines"
        )
    with col2:
        avg_medicines = ingredient_df['MEDICINE_COUNT'].mean()
        st.metric(
            "📊 Average Usage",
            f"{avg_medicines:.0f} medicines",
            "Per ingredient"
        )
    with col3:
        total_unique = len(ingredient_df)
        st.metric(
            "🧪 Unique Ingredients",
            total_unique,
            "In top 30"
        )

with tab2:
    # Scatter plot: Popularity vs Quality
    fig5 = px.scatter(
        ingredient_df,
        x='MEDICINE_COUNT',
        y='AVG_EXCELLENT_REVIEW_PCT',
        size='INGREDIENT_QUALITY_SCORE',
        color='INGREDIENT_QUALITY_SCORE',
        hover_data=['INGREDIENT_NAME', 'DOSAGE', 'BEST_MEDICINE'],
        title='Ingredient Popularity vs Quality Performance',
        labels={
            'MEDICINE_COUNT': 'Number of Medicines Using This Ingredient',
            'AVG_EXCELLENT_REVIEW_PCT': 'Average Excellent Review %',
            'INGREDIENT_QUALITY_SCORE': 'Quality Score'
        },
        color_continuous_scale='RdYlGn'
    )
    
    fig5.update_layout(height=500)
    st.plotly_chart(fig5, use_container_width=True)
    
    st.markdown("**💡 Insight**: This shows if popular ingredients also perform well, or if niche ingredients have higher quality.")
    
    # Show detailed data table
    with st.expander("📋 View Detailed Ingredient Data"):
        st.dataframe(
            ingredient_df[[
                'POPULARITY_RANK', 'INGREDIENT_NAME', 'DOSAGE',
                'MEDICINE_COUNT', 'AVG_EXCELLENT_REVIEW_PCT',
                'INGREDIENT_QUALITY_SCORE', 'BEST_MEDICINE'
            ]].style.format({
                'AVG_EXCELLENT_REVIEW_PCT': '{:.2f}%',
                'INGREDIENT_QUALITY_SCORE': '{:.4f}'
            }),
            use_container_width=True
        )

st.markdown("---")

# ================================================================
# FOOTER
# ================================================================

st.markdown("### 📊 Dashboard Summary")
st.markdown("""
This dashboard provides three key business insights:

1. **Manufacturer Performance**: Identifies which pharmaceutical companies produce the highest-quality medicines
2. **Top Medicines**: Ranks medicines by customer satisfaction to guide promotion and stocking decisions  
3. **Ingredient Analysis**: Shows which active ingredients are most popular and how they perform

**Data Source**: Snowflake Gold Layer (DB_TEAM_HNP.GOLD schema)  
**Update Frequency**: Dynamic tables refresh every 1 hour
""")

st.markdown("---")
st.markdown("*MIS 381N Information Management - University of Texas at Austin - McCombs School of Business*")
'''

Incremental Data Population 

In [ ]:
SELECT COUNT(*) AS rows_before_load
FROM Bronze.DrugDatabase_Raw;

-- ============================================================================
-- STEP 1: CREATE STAGING TABLE
-- ============================================================================

CREATE OR REPLACE TABLE Bronze.DrugDatabase_Staging (
    Medicine_Name VARCHAR,
    Composition VARCHAR,
    Uses VARCHAR,
    Side_Effects VARCHAR,
    Image_URL VARCHAR,
    Manufacturer VARCHAR,
    Excellent_Review_Percentage NUMBER,
    Average_Review_Percentage NUMBER,
    Poor_Review_Percentage NUMBER
);

-- ============================================================================
-- STEP 2: LOAD CSV INTO STAGING TABLE
-- ============================================================================

COPY INTO Bronze.DrugDatabase_Staging
FROM @IncrementalLoad/Sample_DrugDatabase.csv
FILE_FORMAT = (TYPE = 'CSV' FIELD_OPTIONALLY_ENCLOSED_BY = '"' SKIP_HEADER = 1);

-- ============================================================================
-- STEP 3: APPEND STAGING DATA TO BRONZE
-- ============================================================================

INSERT INTO Bronze.DrugDatabase_Raw 
SELECT *
FROM Bronze.DrugDatabase_Staging;

-- ============================================================================
-- STEP 4: ROW COUNT AFTER LOAD
-- ============================================================================

SELECT COUNT(*) AS rows_after_load
FROM Bronze.DrugDatabase_Raw;

--============================================================================
-- STEP 5: CLEANUP
--============================================================================

TRUNCATE TABLE Bronze.DrugDatabase_Staging;

AI SQL Functions

In [ ]:
--============================================================================
-- AI QUERY #1 and #2: Medicine Recommendations & Summarize Medicine        Information
--============================================================================

USE ROLE ROLE_TEAM_HNP;
USE DATABASE DB_TEAM_HNP;
USE SCHEMA BRONZE;
USE WAREHOUSE ANIMAL_TASK_WH;

-- STEP 1: Add columns to table (one-time, free)
ALTER TABLE DRUGDATABASE_RAW 
ADD COLUMN AI_RECOMMENDATION VARCHAR(5000);

ALTER TABLE DRUGDATABASE_RAW 
ADD COLUMN AI_SUMMARY VARCHAR(5000);

-- STEP 2: Update first 10 medicines (test, cheap)
UPDATE DRUGDATABASE_RAW
SET AI_RECOMMENDATION = SNOWFLAKE.CORTEX.COMPLETE(
    'llama3-8b',
    CONCAT(
        'Based on this medicine:\n',
        'Name: ', MEDICINE_NAME, '\n',
        'Ingredients: ', COMPOSITION, '\n',
        'Uses: ', USES, '\n',
        'Side Effects: ', SIDE_EFFECTS, '\n',
        'Rating: ', EXCELLENT_REVIEW_PERCENTAGE, '%\n\n',
        'Give a 2-sentence recommendation for doctors.'
    )
)
WHERE MEDICINE_NAME IN (
    SELECT MEDICINE_NAME 
    FROM DRUGDATABASE_RAW 
    LIMIT 10
);

UPDATE DRUGDATABASE_RAW
SET AI_SUMMARY = SNOWFLAKE.CORTEX.SUMMARIZE(
    CONCAT(
        'Medicine: ', MEDICINE_NAME, '. ',
        'Manufacturer: ', MANUFACTURER, '. ',
        'Ingredients: ', COMPOSITION, '. ',
        'Uses: ', USES, '. ',
        'Side Effects: ', SIDE_EFFECTS, '. ',
        'Rating: ', EXCELLENT_REVIEW_PERCENTAGE, '% excellent reviews.'
    )
)
WHERE MEDICINE_NAME IN (
    SELECT MEDICINE_NAME 
    FROM DRUGDATABASE_RAW 
    LIMIT 10
);

-- STEP 3: View results
SELECT 
    MEDICINE_NAME,
    AI_RECOMMENDATION,
    AI_SUMMARY
FROM DRUGDATABASE_RAW
WHERE AI_RECOMMENDATION IS NOT NULL
LIMIT 10;

AI Cortex Analyst

In [ ]:
--============================================================================
-- QUERY 1: Which medicines should we recommend to patients with bacterial infections?
-- Business Value: Clinical decision support, patient safety, treatment optimization
--============================================================================
USE ROLE ROLE_TEAM_HNP;
USE DATABASE DB_TEAM_HNP;
USE SCHEMA SILVER;
USE WAREHOUSE ANIMAL_TASK_WH;

SELECT 
    MEDICINE_NAME,
    MANUFACTURER,
    EXCELLENT_REVIEW_PERCENTAGE,
    SNOWFLAKE.CORTEX.COMPLETE(
        'llama3-8b',
        CONCAT(
            'Based on this data, should doctors prescribe this for bacterial infections?\n',
            'Medicine: ', MEDICINE_NAME, '\n',
            'Uses: ', USES, '\n',
            'Side Effects: ', SIDE_EFFECTS, '\n',
            'Patient Rating: ', EXCELLENT_REVIEW_PERCENTAGE, '%\n\n',
            'Answer: YES or NO, then give 1 sentence explaining why.'
        )
    ) AS prescription_recommendation
FROM DRUGDATABASE_RAW
WHERE LOWER(USES) LIKE '%bacterial%'
ORDER BY EXCELLENT_REVIEW_PERCENTAGE DESC
LIMIT 10;

In [ ]:
--============================================================================
-- QUERY 2: How do patients perceive different treatment purposes (sentiment analysis)?
-- Business Value: Marketing insights, patient satisfaction, brand positioning
--============================================================================

SELECT 
    MEDICINE_NAME,
    USES,
    SNOWFLAKE.CORTEX.SENTIMENT(USES) AS use_sentiment_score,
    CASE 
        WHEN SNOWFLAKE.CORTEX.SENTIMENT(USES) > 0.5 THEN '😊 Positive'
        WHEN SNOWFLAKE.CORTEX.SENTIMENT(USES) > -0.5 THEN '😐 Neutral'
        ELSE '😞 Negative'
    END AS sentiment_rating,
    EXCELLENT_REVIEW_PERCENTAGE
FROM DRUGDATABASE_RAW
ORDER BY use_sentiment_score DESC
LIMIT 20;

In [ ]:
-- ============================================================================
-- QUERY 3: What are the safest medicines in our database (risk assessment)?
-- Business Value: Risk management, insurance optimization, patient safety
-- ============================================================================

SELECT 
    MEDICINE_NAME,
    MANUFACTURER,
    SIDE_EFFECTS,
    EXCELLENT_REVIEW_PERCENTAGE,
    POOR_REVIEW_PERCENTAGE,
    SNOWFLAKE.CORTEX.COMPLETE(
        'llama3-8b',
        CONCAT(
            'Assess the safety profile:\n',
            'Medicine: ', MEDICINE_NAME, '\n',
            'Side Effects: ', SIDE_EFFECTS, '\n',
            'Excellent Reviews: ', EXCELLENT_REVIEW_PERCENTAGE, '%\n',
            'Poor Reviews: ', POOR_REVIEW_PERCENTAGE, '%\n\n',
            'Rate safety as: VERY SAFE, SAFE, MODERATE RISK, or HIGH RISK. ',
            'Explain in 1 sentence.'
        )
    ) AS safety_rating
FROM DRUGDATABASE_RAW
WHERE EXCELLENT_REVIEW_PERCENTAGE >= 50
ORDER BY POOR_REVIEW_PERCENTAGE ASC
LIMIT 15;

Cortex Search

In [ ]:
USE ROLE ROLE_TEAM_HNP;
USE DATABASE DB_TEAM_HNP;
USE SCHEMA BRONZE;
USE WAREHOUSE ANIMAL_TASK_WH;

CREATE OR REPLACE CORTEX SEARCH SERVICE BRONZE_SEARCH
ON MEDICINE_NAME
WAREHOUSE = ANIMAL_TASK_WH
TARGET_LAG = '1 minute'
AS (
    SELECT 
        MEDICINE_NAME,
        COMPOSITION,
        USES,
        SIDE_EFFECTS,
        MANUFACTURER,
        EXCELLENT_REVIEW_PERCENTAGE,
        AVERAGE_REVIEW_PERCENTAGE,
        POOR_REVIEW_PERCENTAGE,
        IMAGE_URL,
        AI_RECOMMENDATION,
        AI_SUMMARY
    FROM DRUGDATABASE_RAW
);

In [ ]:
-- Search 1: Find antibiotics for bacterial infections
WITH search_results AS (
    SELECT PARSE_JSON(
        SYSTEM$CORTEX_SEARCH_QUERY(
            'DB_TEAM_HNP.BRONZE.BRONZE_SEARCH',
            '{"query": "antibiotics for bacterial infections", "limit": 10}'
        )
    ) AS result
)
SELECT 
    f.value:MEDICINE_NAME::STRING AS medicine_name,
    f.value:"@scores":cosine_similarity::FLOAT AS relevance_score
FROM search_results,
LATERAL FLATTEN(input => result:results) f
ORDER BY relevance_score DESC;

In [ ]:
-- Search 2: Find pain relief for headache and fever
WITH search_results AS (
    SELECT PARSE_JSON(
        SYSTEM$CORTEX_SEARCH_QUERY(
            'DB_TEAM_HNP.BRONZE.BRONZE_SEARCH',
            '{"query": "pain relief headache fever", "limit": 10}'
        )
    ) AS result
)
SELECT 
    f.value:MEDICINE_NAME::STRING AS medicine_name,
    f.value:"@scores":cosine_similarity::FLOAT AS relevance_score
FROM search_results,
LATERAL FLATTEN(input => result:results) f
ORDER BY relevance_score DESC;

In [ ]:
-- Search 3: Find diabetes treatment medication
WITH search_results AS (
    SELECT PARSE_JSON(
        SYSTEM$CORTEX_SEARCH_QUERY(
            'DB_TEAM_HNP.BRONZE.BRONZE_SEARCH',
            '{"query": "diabetes treatment medication", "limit": 10}'
        )
    ) AS result
)
SELECT 
    f.value:MEDICINE_NAME::STRING AS medicine_name,
    f.value:"@scores":cosine_similarity::FLOAT AS relevance_score
FROM search_results,
LATERAL FLATTEN(input => result:results) f
ORDER BY relevance_score DESC;